<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>

Aluno: [Mateus Cardoso](https://www.linkedin.com/in/mateusgcs/)

Professor: André Perez

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **COVID Dashboard**

## 1\. Contexto

Este é um caderno de preparação de dados original para alimentar um painel no Google Data Studio, desenvolvido para o curso de Análise de Dados da EBAC, criado em maio de 2024.

 - **Dashboard**: Google Data Studio ([link](https://lookerstudio.google.com/reporting/730fb34a-8cbf-4566-98bd-342e8802dd2b)).
 - **Processamento**: Kaggle Notebook (`link`).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

### **1.2. Pandemia Coronavírus 2019**

A COVID-19 é uma condição respiratória aguda, derivada do coronavírus SARS-CoV-2, caracterizada por sua severidade, alta taxa de transmissão e alcance global. Informações fornecidas pelo governo brasileiro.

### **1.3. Dados**

As informações referentes aos **casos de COVID-19** são compiladas pelo Centro de Ciência de Sistemas e Engenharia da Universidade Johns Hopkins, situada nos Estados Unidos ([link](https://www.jhu.edu)). Desde janeiro de 2020, esses dados são atualizados diariamente, com detalhamento temporal por dias e geográfico por regiões de países (tais como estados, condados, etc.). O projeto disponibiliza seu website através deste [link](https://systems.jhu.edu/research/public-health/ncov/) e os dados podem ser acessados diretamente por meio deste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão apresentadas as informações derivadas do processamento desses dados.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

As informações relacionadas à **vacinação contra a COVID-19** são compiladas pelo projeto Our World in Data (OWID) da Universidade de Oxford, no Reino Unido ([link](https://www.ox.ac.uk)). Desde janeiro de 2020, esses dados são **atualizados diariamente** e possuem uma **granularidade temporal por dias e geográfica por países**. O website do projeto pode ser acessado através deste [link](https://ourworldindata.org), enquanto os dados podem ser obtidos diretamente neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). A seguir, estão descritas as informações resultantes do processamento desses dados.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

---

## 2\. Pacotes e bibliotecas

In [1]:
from datetime import datetime, timedelta
import math
from typing import Iterator
import numpy as np
import pandas as pd


---

## 3\. Casos

### **3.2. Extração dos dados de Casos**

Extração de dados de casos fornecidos pela universidade John Hopkins.




In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [3]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


In [4]:
cases.tail()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
4007,NaN,NaN,Unknown,Ukraine,2021-01-13 05:22:15,NaN,NaN,0,0,0,0,"Unknown, Ukraine",0.0,0.0
4008,NaN,NaN,NaN,Nauru,2021-01-13 05:22:15,-0.5228,166.9315,0,0,0,0,Nauru,0.0,0.0
4009,NaN,NaN,Niue,New Zealand,2021-01-13 05:22:15,-19.0544,-169.8672,0,0,0,0,"Niue, New Zealand",0.0,0.0
4010,NaN,NaN,NaN,Tuvalu,2021-01-13 05:22:15,-7.1095,177.6493,0,0,0,0,Tuvalu,0.0,0.0
4011,NaN,NaN,Pitcairn Islands,United Kingdom,2021-01-13 05:22:15,-24.3768,-128.3242,0,0,0,0,"Pitcairn Islands, United Kingdom",0.0,0.0


- Como demonstrado acima, a granularidade entre arquivos está de forma diária, então no próximo passo, vamos iterar dentro de um intervalo de tempo especifico e filtrar as informações para voltar a análise para o Brasil.

In [5]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [6]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [7]:
cases = None

cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')

  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)

  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], ignore_index=True)

### **3.3. Wrangling dos dados - Ocorrências**

Vamos manipular os dados para facilitar a leitura e interpretação das informações.

In [8]:
cases.shape

(9828, 6)

In [9]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [10]:
cases.tail()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
9823,Roraima,Brazil,128793,2078,21261.355551,2021-12-30
9824,Santa Catarina,Brazil,1242654,20183,17343.904663,2021-12-30
9825,Sao Paulo,Brazil,4455011,155186,9701.879932,2021-12-30
9826,Sergipe,Brazil,278507,6057,12115.869171,2021-12-30
9827,Tocantins,Brazil,235558,3939,14976.355265,2021-12-30


In [11]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province_State  9828 non-null   object        
 1   Country_Region  9828 non-null   object        
 2   Confirmed       9828 non-null   int64         
 3   Deaths          9828 non-null   int64         
 4   Incident_Rate   9828 non-null   float64       
 5   Date            9828 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 460.8+ KB


In [12]:
cases[['Province_State', 'Country_Region']].describe()

,Province_State,Country_Region
count,9828,9828
unique,27,1
top,Acre,Brazil
freq,364,9828


In [13]:
cases[['Confirmed', 'Deaths', 'Incident_Rate']].describe()

,Confirmed,Deaths,Incident_Rate
count,9.828000e+03,9828.000000,9828.000000
mean,6.294380e+05,17191.270350,9481.681339
std,7.174642e+05,24220.203084,3954.420024
min,4.168900e+04,787.000000,2333.277633
25%,2.301745e+05,5095.000000,6384.567813
50%,3.829510e+05,9623.000000,9391.641823
75%,7.598618e+05,19137.750000,12282.871338
max,4.455011e+06,155186.000000,21261.355551


In [14]:
cases['Date'].describe()

count                             9828
mean     2021-07-01 11:59:59.999999744
min                2021-01-01 00:00:00
25%                2021-04-01 18:00:00
50%                2021-07-01 12:00:00
75%                2021-09-30 06:00:00
max                2021-12-30 00:00:00
Name: Date, dtype: object

- Renomeando colunas e padronizando as informações contidas no dataframe:

In [15]:
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

In [16]:
for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [17]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

- Adição de novas colunas para enriquecer as informações fornecidas.



1. Chaves temporais nas colunas year e month



In [18]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

2. População estimada dos estados.

In [19]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))

cases = cases.drop('incident_rate', axis=1)

3. Número diário, média móvel e estabilidade

In [20]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status

for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state], ignore_index=True)
cases = cases_
cases_ = None

In [21]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [22]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [23]:
cases.shape

(9828, 16)

In [24]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9828 entries, 0 to 9827
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date                              9828 non-null   datetime64[ns]
 1   country                           9828 non-null   object        
 2   state                             9828 non-null   object        
 3   population                        9828 non-null   Int64         
 4   confirmed                         9828 non-null   int64         
 5   confirmed_1d                      9801 non-null   Int64         
 6   confirmed_moving_avg_7d           9639 non-null   Int64         
 7   confirmed_moving_avg_7d_rate_14d  9261 non-null   float64       
 8   confirmed_trend                   9261 non-null   object        
 9   deaths                            9828 non-null   int64         
 10  deaths_1d                         9801 non-null 

In [25]:
cases.head()

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021


In [26]:
cases.tail()

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
9823,2021-12-26,Brazil,Tocantins,1572866,234113,0,0,0.000000,downward,3927,0,0,0.0,downward,2021-12,2021
9824,2021-12-27,Brazil,Tocantins,1572866,234113,0,0,0.000000,downward,3927,0,0,0.0,downward,2021-12,2021
9825,2021-12-28,Brazil,Tocantins,1572866,234964,851,122,2.837209,upward,3933,6,1,1.0,stable,2021-12,2021
9826,2021-12-29,Brazil,Tocantins,1572866,235340,376,176,inf,upward,3936,3,2,inf,upward,2021-12,2021
9827,2021-12-30,Brazil,Tocantins,1572866,235558,218,207,inf,upward,3939,3,2,inf,upward,2021-12,2021


In [27]:
cases[['population', 'confirmed_1d', 'confirmed_moving_avg_7d', 'deaths_1d', 'deaths_moving_avg_7d']].describe()

,population,confirmed_1d,confirmed_moving_avg_7d,deaths_1d,deaths_moving_avg_7d
count,9828.0,9801.0,9639.0,9801.0,9639.0
mean,7787582.103073,1487.366391,1500.036311,43.231099,44.149082
std,9078237.503882,2890.408447,2304.293046,94.524523,81.357065
min,605761.0,-1582.0,-108.0,-38.0,0.0
25%,2792910.25,190.0,244.5,4.0,5.0
50%,4019300.0,630.0,732.0,14.0,17.0
75%,9557071.0,1565.0,1631.5,39.0,46.0
max,46217709.0,105200.0,18217.0,1389.0,891.0


---

### **3.4. Carregamento dos dados - Ocorrências**

In [28]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

## 4\. Vacinação

### **4.2. Extração dos dados - Vacinação**

Vamos extrair os dados de **vacinação** da universidade de Oxford.

In [29]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

<ipython-input-29-bcbb2df5cfca>:3: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


In [30]:
vaccines.shape

(388920, 67)

In [31]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [32]:
vaccines.tail()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
388915,ZWE,Africa,Zimbabwe,2024-03-20,266359.0,0.0,0.0,5740.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
388916,ZWE,Africa,Zimbabwe,2024-03-21,266359.0,0.0,0.0,5740.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
388917,ZWE,Africa,Zimbabwe,2024-03-22,266359.0,0.0,0.0,5740.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
388918,ZWE,Africa,Zimbabwe,2024-03-23,266359.0,0.0,0.0,5740.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN
388919,ZWE,Africa,Zimbabwe,2024-03-24,266359.0,0.0,0.0,5740.0,0.0,0.0,...,30.7,36.791,1.7,61.49,0.571,16320539.0,NaN,NaN,NaN,NaN


- Selecionando colunas de interesse para analisar somente o caso do Brasil:

In [33]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [34]:
vaccines.shape

(1541, 7)

In [35]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-09


In [36]:
vaccines.tail()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
1536,Brazil,215313504.0,NaN,NaN,NaN,NaN,2024-03-20
1537,Brazil,215313504.0,NaN,NaN,NaN,NaN,2024-03-21
1538,Brazil,215313504.0,NaN,NaN,NaN,NaN,2024-03-22
1539,Brazil,215313504.0,NaN,NaN,NaN,NaN,2024-03-23
1540,Brazil,215313504.0,NaN,NaN,NaN,NaN,2024-03-24


---

### **4.3. Wrangling dos dados - Vacinação**

In [37]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1541 entries, 0 to 1540
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 1541 non-null   object        
 1   population               1541 non-null   float64       
 2   total_vaccinations       695 non-null    float64       
 3   people_vaccinated        691 non-null    float64       
 4   people_fully_vaccinated  675 non-null    float64       
 5   total_boosters           455 non-null    float64       
 6   date                     1541 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 84.4+ KB


In [38]:
vaccines['location'].describe()

count       1541
unique         1
top       Brazil
freq        1541
Name: location, dtype: object

In [39]:
vaccines[['population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters']].describe()

,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters
count,1541.0,6.950000e+02,6.910000e+02,6.750000e+02,4.550000e+02
mean,215313504.0,2.905069e+08,1.343343e+08,1.119384e+08,7.455898e+07
std,0.0,1.744321e+08,6.595258e+07,6.574517e+07,4.565270e+07
min,215313504.0,1.120000e+02,1.120000e+02,1.962000e+03,2.820000e+02
25%,215313504.0,1.134644e+08,8.455755e+07,3.731281e+07,2.548890e+07
50%,215313504.0,3.313358e+08,1.664270e+08,1.454594e+08,8.790341e+07
75%,215313504.0,4.615308e+08,1.850926e+08,1.709911e+08,1.187733e+08
max,215313504.0,4.864364e+08,1.896434e+08,1.761642e+08,1.263886e+08


In [40]:
vaccines['date'].describe()

count                             1541
mean     2022-02-12 23:59:59.999999744
min                2020-01-05 00:00:00
25%                2021-01-24 00:00:00
50%                2022-02-13 00:00:00
75%                2023-03-05 00:00:00
max                2024-03-24 00:00:00
Name: date, dtype: object

- Tratamento de dados faltantes, a estratégia será a de preencher os buracos com o valor anterior válido mais próximo:

In [41]:
vaccines = vaccines.fillna(method='ffill')

- Filtrando o dataframe para que as informações contidas estejam no mesmo padrão de período que o arquivo de ocorrências:

In [42]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

- Vamos também alterar os nomes das colunas para que representem melhor os dados que contém:

In [43]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

1. Chaves temporais

In [44]:
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

2. Dados relativos

In [45]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)

vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)

vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [46]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

- Cumpridas todas essas etapas, vamos reorganizar as colunas para facilitar a visualização e conferir o resultado final do wrangling

In [47]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [48]:
vaccines.shape

(365, 12)

In [49]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              365 non-null    datetime64[ns]
 1   country           365 non-null    object        
 2   population        365 non-null    Int64         
 3   total             349 non-null    Int64         
 4   one_shot          349 non-null    Int64         
 5   one_shot_perc     349 non-null    float64       
 6   two_shots         330 non-null    Int64         
 7   two_shots_perc    330 non-null    float64       
 8   three_shots       121 non-null    Int64         
 9   three_shots_perc  121 non-null    float64       
 10  month             365 non-null    object        
 11  year              365 non-null    object        
dtypes: Int64(5), datetime64[ns](1), float64(3), object(3)
memory usage: 36.1+ KB


In [50]:
vaccines.head()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
0,2021-01-01,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
1,2021-01-02,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
2,2021-01-03,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
3,2021-01-04,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021
4,2021-01-05,Brazil,215313504,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,2021-01,2021


In [51]:
vaccines.tail()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
360,2021-12-27,Brazil,215313504,329011365,165952037,0.7707,142764283,0.6631,25218893,0.1171,2021-12,2021
361,2021-12-28,Brazil,215313504,329861730,166062249,0.7713,142965728,0.6640,25758909,0.1196,2021-12,2021
362,2021-12-29,Brazil,215313504,330718457,166143380,0.7716,143282084,0.6655,26219623,0.1218,2021-12,2021
363,2021-12-30,Brazil,215313504,331164041,166185628,0.7718,143398692,0.6660,26507937,0.1231,2021-12,2021
364,2021-12-31,Brazil,215313504,331273910,166195505,0.7719,143436012,0.6662,26571077,0.1234,2021-12,2021


In [52]:
vaccines[['one_shot_perc', 'two_shots_perc', 'three_shots_perc']].describe()

,one_shot_perc,two_shots_perc,three_shots_perc
count,349.000000,330.000000,121.000000
mean,0.399109,0.253927,0.046561
std,0.288012,0.224663,0.039909
min,0.000000,0.000000,0.000000
25%,0.116100,0.068625,0.006500
50%,0.399100,0.161250,0.041200
75%,0.710300,0.461675,0.078300
max,0.771900,0.666200,0.123400


### **4.4. Carregamento dos dados de Vacinação**

In [53]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

In [56]:
vaccines.tail()

,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,month,year
360,2021-12-27,Brazil,215313504,329011365,165952037,0.7707,142764283,0.6631,25218893,0.1171,2021-12,2021
361,2021-12-28,Brazil,215313504,329861730,166062249,0.7713,142965728,0.6640,25758909,0.1196,2021-12,2021
362,2021-12-29,Brazil,215313504,330718457,166143380,0.7716,143282084,0.6655,26219623,0.1218,2021-12,2021
363,2021-12-30,Brazil,215313504,331164041,166185628,0.7718,143398692,0.6660,26507937,0.1231,2021-12,2021
364,2021-12-31,Brazil,215313504,331273910,166195505,0.7719,143436012,0.6662,26571077,0.1234,2021-12,2021
